# **imports**

In [0]:
import os
import struct
import numpy as np
from time import time
from numpy import random as rnd
import matplotlib.pyplot as plt
import matplotlib.pyplot as pyplot
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from IPython.display import clear_output
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# **Download MNIST Files** (working at google colaboratory)

In [0]:
# !wget http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
# !wget http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
# !wget http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
# !wget http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz

# !gzip -d train-images.idx3-ubyte.gz
# !gzip -d train-labels.idx1-ubyte.gz
# !gzip -d t10k-images.idx3-ubyte.gz
# !gzip -d t10k-labels.idx1-ubyte.gz
# clear_output()
# print()
# #!ls

In [0]:
"""
Loosely inspired by http://abel.ee.ucla.edu/cvxopt/_downloads/mnist.py
which is GPL licensed.
"""
def read(dataset = "training", path = "C:/datas/"):
    """
    Python function for importing the MNIST data set.  It returns an iterator
    of 2-tuples with the first element being the label and the second element
    being a numpy.uint8 2D array of pixel data for the given image.
    """

    if dataset is "training":
        fname_img = os.path.join(path, 'train-images.idx3-ubyte')
        fname_lbl = os.path.join(path, 'train-labels.idx1-ubyte')
        print('read() training!!')
    elif dataset is "testing":
        fname_img = os.path.join(path, 't10k-images.idx3-ubyte')
        fname_lbl = os.path.join(path, 't10k-labels.idx1-ubyte')
        print('read() testing!!')
    else:
        raise Exception("dataset must be 'testing' or 'training'")

    # Load everything in some numpy arrays
    with open(fname_lbl, 'rb') as flbl:
        magic, num = struct.unpack(">II", flbl.read(8))
        lbl = np.fromfile(flbl, dtype=np.int8)

    with open(fname_img, 'rb') as fimg:
        magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        img = np.fromfile(fimg, dtype=np.uint8).reshape(len(lbl), rows, cols)

    get_img = lambda idx: (lbl[idx], img[idx])

    # Create an iterator which returns each image in turn
    for i in range(len(lbl)):
        yield get_img(i)

def show(image):
    """
    Render a given numpy.uint8 2D array of pixel data.
    """
    from matplotlib import pyplot
    import matplotlib as mpl
    fig = pyplot.figure()
    ax = fig.add_subplot(1,1,1)
    imgplot = ax.imshow(image, cmap=mpl.cm.Greys)
    imgplot.set_interpolation('nearest')
    ax.xaxis.set_ticks_position('top')
    ax.yaxis.set_ticks_position('left')
read()
pyplot.show()

# **Reading training set**

In [3]:
# read dataset fron file.

#tr = list(read("training", "/content/gdrive/My Drive/"))
#ts = list(read("testing","/content/gdrive/My Drive/"))
tr = list(read("training", "C:/datas"))
ts = list(read("testing", "C:/datas"))


LIMIT = 5000
X_train = np.array(list(zip(*tr))[1][:LIMIT])
y_train = np.array(list(zip(*tr))[0][:LIMIT])

X_test = np.array(list(zip(*ts))[1][:LIMIT])
y_test = np.array(list(zip(*ts))[0][:LIMIT])

#print(X_train.shape)
#print(y_train.shape)
#print(X_test.shape)
#print(y_test.shape)

read() training!!
read() testing!!


# **Split Set**

In [0]:
def split_set(d = None):
  if d == None:
    return train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
  elif d == 'scale':
    return train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)
  
  print(X_train.shape)

# **Compare [Non-scaled, Scaled] sets**

In [5]:
print(X_train.shape)
print(y_train.shape)

print(y_train.shape)
print(y_test.shape)

(5000, 28, 28)
(5000,)
(5000,)
(5000,)


In [12]:
training_size = len(X_train)
testing_size = len(X_test)
X_train = X_train.reshape(training_size, 784)
X_test = X_test.reshape(testing_size, 784)
clf = LogisticRegression(C=50,
                         multi_class='multinomial',
                         penalty='l2', solver='saga', tol=0.1)

clf.fit(X_train, y_train)
clear_output()
y_predict = clf.predict(X_test)

print('Training size : %d Testing Size : %d' %(training_size, testing_size))
print(f'normal score: {accuracy_score(y_test,clf.predict(X_test))}')

y_predict = clf.predict(X_test)
# cm = confusion_matrix(y_test, y_predict)
print(classification_report(y_test, y_predict, 
                            target_names=['0','1','2','3','4','5','6','7','8','9']))


# print(f'normal score: {accuracy_score(y_test, clf.predict(X_test))}')

Training size : 5000 Testing Size : 5000
normal score: 0.8754
              precision    recall  f1-score   support

           0       0.92      0.97      0.94       460
           1       0.92      0.98      0.95       571
           2       0.90      0.83      0.86       530
           3       0.86      0.86      0.86       500
           4       0.86      0.91      0.88       500
           5       0.84      0.81      0.83       456
           6       0.92      0.87      0.89       462
           7       0.88      0.84      0.86       512
           8       0.84      0.81      0.83       489
           9       0.82      0.86      0.84       520

    accuracy                           0.88      5000
   macro avg       0.88      0.87      0.87      5000
weighted avg       0.88      0.88      0.87      5000



# **After Preprocessing Do LR**

In [13]:
training_size = len(preprocessed_1_X_train)
testing_size = len(preprocessed_1_X_test)
size_A = len(X_train[0])
preprocessed_1_X_train = X_train.reshape(training_size, size_A)
preprocessed_1_X_test = X_test.reshape(testing_size, size_A)

clf_A = LogisticRegression(C=50,
                         multi_class='multinomial',
                         penalty='l2', solver='saga', tol=0.1)

clf_A.fit(preprocessed_1_X_train, y_train)
clear_output()
y_predict = clf.predict(preprocessed_1_X_test)

print(f'normal score: {accuracy_score(y_test,clf.predict(preprocessed_1_X_test))}')

# print(f'normal score: {accuracy_score(y_test, clf.predict(X_test))}')

NameError: ignored

In [0]:
y_predict = clf_A.predict(preprocessed_1_X_test)
cm = confusion_matrix(y_test, y_predict)

print(cm)
print(classification_report(y_test, y_predict, 
                            target_names=['0','1','2','3','4','5','6','7','8','9']))

[[ 958    0    4    8    3    6    8    3   10    1]
 [   0 1098    5    2    2   10    0    4    4    2]
 [   8   13  870   11   24    6   13   16   24    6]
 [   1    9   30  905    2   31    6   14   12   22]
 [   3    7    9    0  914    1    7    2    4   33]
 [   7   10    4   41   19  720   19    1   30   12]
 [   7    4    9    0    6   13  968    0    6    1]
 [   3    7    9    3   17    3    0  977    5   46]
 [   6   27   21   25    3   39   22    5  784   12]
 [  10    4    8   18   21    5    0   29    8  875]]
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1001
           1       0.93      0.97      0.95      1127
           2       0.90      0.88      0.89       991
           3       0.89      0.88      0.89      1032
           4       0.90      0.93      0.92       980
           5       0.86      0.83      0.85       863
           6       0.93      0.95      0.94      1014
           7       0.93      0.91   

In [0]:
training_size = len(X_train)
X_train = X_train.reshape(training_size, 784)
X_test = X_test.reshape(training_size, 784)
clf = LogisticRegression(C=50 / training_size,
                         multi_class='multinomial',
                         penalty='l2', solver='saga', tol=0.1)

# clf_s = LogisticRegression(C=50 / training_size ,
#                          multi_class='multinomial',
#                          penalty='l2', solver='saga', tol=0.1)

# X_train, X_test, y_train, y_test = split_set()

clf.fit(X_train, y_train)
clear_output()
print(f'after preprocessed score: {accuracy_score(y_test,clf.predict(X_test))}')

# scaler = StandardScaler()
# scaler.fit(X)
# X_scaled = scaler.transform(X)

# X_train_scaled, X_test, y_train, y_test = split_set('scale')

# clf_s.fit(X_train_scaled, y_train)
# print(f'standard scaled score: {accuracy_score(y_test,clf.predict(X_test))}')

normal score: 0.9088


In [0]:
training_size_1 = len(X_svc_grads_train)
testing_size_1 = len(X_svc_grads_test)
#X_train = X_train.reshape(training_size_1, 784)
#X_test = X_test.reshape(testing_size_1, 784)

clf = LogisticRegression(C=50,
                         multi_class='multinomial',
                         penalty='l2', solver='saga', tol=0.1)

clf.fit(X_svc_grads_train, y_train)
clear_output()

print(f'normal score: {accuracy_score(y_test,clf.predict(X_svc_grads_test))}')

# print(f'normal score: {accuracy_score(y_test, clf.predict(X_test))}')

normal score: 0.0865


In [0]:
print(X_svc_grads_train.shape)
print(X_svc_grads_test.shape)

# **Preprocessing**

In [0]:
def train_model(X, Y, model, default_param, param_grid, cv, save, filename):
    
    clf = model(**default_param)
    estimator = GridSearchCV(clf, param_grid, cv=cv, n_jobs=-1)
    
    estimator.fit(X, Y)
    
    if save:
        joblib.dump(estimator, filename)
        
    return estimator

def get_model(model, filename, param_grid=dict(), cv=6, X=None, Y=None, default_param=dict(), force_training=False, save=True):
    
    path = pathlib.Path(filename)
    estimator = None
    
    if force_training == True:
        estimator = train_model(X, Y, model, default_param, param_grid, cv, save, filename)
    else:
        if path.exists(): # if file exists, just load that.
            estimator = joblib.load(filename)
        else:
            estimator = train_model(X, Y, model, default_param, param_grid, cv, save, filename)
            
    return estimator

def get_model_without_gridsearch(model, filename, default_param=dict(), X=None, Y=None, save=True):
    
    if pathlib.Path(filename).exists():
        estimator = joblib.load(filename)
    else:
        estimator = model(**default_param)
        estimator.fit(X, Y)
        if save:
            joblib.dump(estimator, filename)
        
    return estimator

# **Zero Padding**

In [0]:
def zero_padding(images, n=1):
    """
    zero-padding to image.
    add additional edge which has value of 0
    
    Arguments:
    ---------------------
    - images: training dataset images. maybe (60000, 28, 28)
    - n: how many padding do you want? in other word, how many edge do you want to insert?
    
    Returns:
    ---------------------
    - images_padded: padded images. (60000, 30, 30) or other shape.
    """
    
    # number of training examples. 60000. if you use test data, 10000.
    m = images.shape[0]
    print(m)
    # define larger size of window than size of images. maybe (60000, 30, 30), (60000, 32, 32)
    images_padded = np.zeros((m, images.shape[1] + 2 * n, images.shape[2] + 2 * n))
    
    # insert image in the middle of this window.
    images_padded[:, n : images_padded.shape[1] - n, n : images_padded.shape[2] - n] = images
    
    return images_padded


In [0]:
def x_gradient_slice(images_slice):
    """
    find gradient(in korean, 기울기 또는 미분값) for part of images.
    
    Arguments:
    ----------------------
    - images_slice: small window extracted from images. (60000, 7, 7)
    
    Returns:
    ----------------------
    - grad: x-axis-oriented gradient (in korean, x 축 방향 기울기)p
    """
    
    x_gradient_filter = np.array([
        [-1,  0,  1],
        [-1,  0,  1],
        [-1,  0,  1],
    ])
    
    # reshape for broadcasting.
    x_gradient_filter = x_gradient_filter.reshape(1, 3, 3)
    
    # element-wise compute. compute gradient
    temp = np.multiply(images_slice, x_gradient_filter)
    grad = np.sum(temp, axis=(1, 2))
    
    return grad

In [0]:
def x_gradient(images):
    
    # some useful variables.
    m = images.shape[0]
    width = images.shape[1]
    height = images.shape[2]
    
    # define placeholder to store gradients.
    x_grads = np.zeros((m, width - 2, height - 2))
    
    # slice image into small size window, then compute gradient.
    for w in range(1, width - 1):
        for h in range(1, height - 1):
            images_slice = images[:, w - 1 : w + 2, h - 1 : h + 2]
            x_grads[:, w - 1, h - 1] = x_gradient_slice(images_slice)
            
    return x_grads

In [0]:
def y_gradient_slice(images_slice):
    """
    find gradient(in korean, 기울기 또는 미분값) for part of images.
    
    Arguments:
    ----------------------
    - images_slice: small window extracted from images. (60000, 7, 7)
    
    Returns:
    ----------------------
    - grad: y-axis-oriented gradient (in korean, y 축 방향 기울기)
    """
    
    y_gradient_filter = np.array([
        [-1, -1, -1],
        [ 0,  0,  0],
        [ 1,  1,  1],
    ])
    
    # reshape for broadcasting.
    y_gradient_filter = y_gradient_filter.reshape(1, 3, 3)
    
    # element-wise compute. compute gradient
    temp = np.multiply(images_slice, y_gradient_filter)
    grad = np.sum(temp, axis=(1, 2))
    
    return grad

In [0]:
def y_gradient(images):
    
    # some useful variables.
    m = images.shape[0]
    width = images.shape[1]
    height = images.shape[2]
    
    # define placeholder to store gradients.
    y_grads = np.zeros((m, width - 2, height - 2))
    
    # slice image into small size window, then compute gradient.
    for w in range(1, width - 1):
        for h in range(1, height - 1):
            images_slice = images[:, w - 1 : w + 2, h - 1 : h + 2]
            y_grads[:, w - 1, h - 1] = y_gradient_slice(images_slice)
            
    return y_grads

In [0]:
def get_average_grads(x_grads, y_grads, grid=7):
    
    
    assert(x_grads.shape == y_grads.shape)
    
    # some useful variables.
    m = x_grads.shape[0]
    width = x_grads.shape[1]
    height = x_grads.shape[2]
    
    # I define these variables to slicing images conveniently.
    w_step = width // grid  # w_step = 4
    h_step = height // grid # h_step = 4
    
    # placeholder for storing average of gradients, 빈공간 만들기
    x_avg_grads = np.zeros((m, width // w_step, height // h_step))
    y_avg_grads = np.zeros((m, width // w_step, height // h_step))
    
    for w in range(0, width, w_step):
        for h in range(0, height, h_step):
            # slicing gradients into small part.
            x_grads_slice = x_grads[:, w : w + w_step, h : h + h_step]
            y_grads_slice = y_grads[:, w : w + w_step, h : h + h_step]
            
            assert(x_grads_slice.shape == y_grads_slice.shape == (m, width // grid, height // grid))
            
            # compute mean of gradients of part of image
            x_avg_grads[:, w // w_step, h // h_step] = np.mean(x_grads_slice, axis=(1, 2))
            y_avg_grads[:, w // w_step, h // h_step] = np.mean(y_grads_slice, axis=(1, 2))
            
    return x_avg_grads, y_avg_grads

In [0]:
def average_gradients_grid(images, grid=7, padding=1, normalize=True):
    
    images = np.copy(images)
    
    m = images.shape[0]
    
    # normalize
    if normalize:
        images_norm = images / 255
    else:
        images_norm = images
    
    # thresholding
    images_norm[images_norm >= 0.2] = 1
    images_norm[images_norm < 0.2] = 0
    
    # zero padding
    images_padded = zero_padding(images_norm, padding)
    
    # number of features = grid^2 * 2
    features = np.zeros((m, (grid ** 2) * 2))

    # compute x-axis gradient, y-axis gradient
    x_grads = x_gradient(images_padded)
    y_grads = y_gradient(images_padded)
    
    # compute average of gradient (grid 7x7)
    x_avg_grads, y_avg_grads = get_average_grads(x_grads, y_grads, grid)
    
    assert(x_avg_grads.shape == y_avg_grads.shape == (m, grid, grid))
    
    # flatten
    x_features = x_avg_grads.reshape(m, -1)
    y_features = y_avg_grads.reshape(m, -1)
    
    features[:, : grid ** 2] = x_features
    features[:, grid ** 2 :] = y_features
    
    return features

In [0]:
X_train = X_train.reshape(training_size, 28, 28)
X_test = X_test.reshape(testing_size, 28, 28)

print(X_train.shape)
print(X_test.shape)

ValueError: ignored

In [0]:
print(X_train.shape)
print(X_test.shape)

(10000, 28, 28)
(30000, 28, 28)


In [0]:
preprocessed_1_X_train = average_gradients_grid(X_train)
preprocessed_1_X_test = average_gradients_grid(X_test)

10000


MemoryError: ignored

In [0]:
print(preprocessed_1_X_train.shape)
print(preprocessed_1_X_test.shape)

# **Preprocessing**

# **New Grid Search**

In [0]:
import pathlib
from sklearn.externals import joblib

param_grid = [
    {
        'penalty': ['l1', 'l2'],
        'C': [1e-4, 1e-4, 1e-3, 1e-2, 1e-1, 1, 1000], # 1000 means no regularization
        'solver':['saga']
    }
]

# load or train model.
clf_logs_1_1 = get_model(LogisticRegression, 'C:/datas/logistic_1.model_data', param_grid, 3, X=preprocessed_1_X_train, Y=y_train, default_param=dict(solver='lbfgs'))

c:\users\김윤기\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
c:\users\김윤기\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\김윤기\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [0]:
param_logs_1_1 = clf_logs_1_1.best_params_
print(param_logs_1_1)

# get best logistic regression we found on grid search. This contain best parameters.
logs_1_1 = clf_logs_1_1.best_estimator_

# get best accuracy on training set
print('Accuracy on training set:')
print(clf_logs_1_1.best_score_)

# compute accuracy on test set using best logistic regression
print('Accuracy on test set:')
print(logs_1_1.score(preprocessed_1_X_test, y_test))

{'C': 1000, 'penalty': 'l1', 'solver': 'saga'}
Accuracy on training set:
0.8851
Accuracy on test set:
0.894


# **End New Grid Search**

# **Grid Search After Preprocessing**



In [0]:
print(X_svc_grads_train.shape)
print(X_svc_grads_test.shape)

(10000, 98)
(10000, 98)


In [0]:
print(preprocessed_1_X_train.shape)
print(preprocessed_1_X_test.shape)

In [0]:
import time

start_time = time.time()
f1=make_scorer(f1_score, average='macro')
params = [{'penalty':['l1','l2'], 'C':[1000, 500, 100, 50, 10], 'solver':['saga']}]
grid_search_clf_processed = GridSearchCV(LogisticRegression(random_state=42, max_iter=30, tol=1e-3),
                  param_grid=params,
                  scoring=f1,
                  cv=5)

# print(grid_search_clf)
grid_search_clf_processed.fit(preprocessed_1_X_train, y_train)
clear_output()
elaspe_time = time.time() - start_time
print('Elapse time :%.1f' % (elaspe_time),'sec')
print('Best Parameter :', grid_search_clf_processed.best_params_)
cv_result_processed = grid_search_clf_processed.cv_results_
for mean_score, params in zip(cv_result_processed['mean_test_score'], cv_result_processed['params']):
  print(mean_score, params)


Elapse time :188.6 sec
Best Parameter : {'C': 50, 'penalty': 'l2', 'solver': 'saga'}
0.8938192641396682 {'C': 1000, 'penalty': 'l1', 'solver': 'saga'}
0.8938192641396682 {'C': 1000, 'penalty': 'l2', 'solver': 'saga'}
0.8938192641396682 {'C': 500, 'penalty': 'l1', 'solver': 'saga'}
0.8938192641396682 {'C': 500, 'penalty': 'l2', 'solver': 'saga'}
0.8938192641396682 {'C': 100, 'penalty': 'l1', 'solver': 'saga'}
0.8938192641396682 {'C': 100, 'penalty': 'l2', 'solver': 'saga'}
0.8938192641396682 {'C': 50, 'penalty': 'l1', 'solver': 'saga'}
0.8938200230751925 {'C': 50, 'penalty': 'l2', 'solver': 'saga'}
0.8933114014517357 {'C': 10, 'penalty': 'l1', 'solver': 'saga'}
0.8932189969677249 {'C': 10, 'penalty': 'l2', 'solver': 'saga'}


In [0]:


X_train, X_test, y_train, y_test = split_set()
print(X_train.shape)


lg_model  = LogisticRegression(C=0.1, penalty='l1', random_state = 77)
lg_model.fit(X_train,y_train)

y_predict = lg_model.predict(X_test)
cm = confusion_matrix(y_test, y_predict)
print(classification_report(y_test, y_predict, 
                            target_names=['0','1','2','3','4','5','6','7','8','9']))

# **Grid Search**

In [0]:
X_train = X_train
y_train = y_train
X_test = X_test
y_test = y_test

In [0]:
start_time_1 = time()
f1_1=make_scorer(f1_score, average='macro')
params = [{'penalty':['l1','l2'], 'C':[10,1,0.1,0.01,0.001], 'solver':['saga']}]
grid_search_clf = GridSearchCV(LogisticRegression(random_state=42, max_iter=30, tol=1e-3),
                  param_grid=params,
                  scoring=f1_1,
                  cv=5)

# print(grid_search_clf)
grid_search_clf.fit(X_train, y_train)


TypeError: ignored

In [0]:
start = time.time()
print('time : %.1f' %(time.time() - start))

time : 0.0


In [0]:
cv_result = grid_search_clf.cv_results_
for mean_score, params in zip(cv_result['mean_test_score'], cv_result['params']):
  print(mean_score, params)
grid_search_clf.best_params_

0.9002521286056621 {'C': 10, 'penalty': 'l1', 'solver': 'saga'}
0.9002521286056621 {'C': 10, 'penalty': 'l2', 'solver': 'saga'}
0.9002521286056621 {'C': 1, 'penalty': 'l1', 'solver': 'saga'}
0.9002521286056621 {'C': 1, 'penalty': 'l2', 'solver': 'saga'}
0.9001470563977063 {'C': 0.1, 'penalty': 'l1', 'solver': 'saga'}
0.9002521286056621 {'C': 0.1, 'penalty': 'l2', 'solver': 'saga'}
0.901636178422903 {'C': 0.01, 'penalty': 'l1', 'solver': 'saga'}
0.9002521286056621 {'C': 0.01, 'penalty': 'l2', 'solver': 'saga'}
0.8949072862116091 {'C': 0.001, 'penalty': 'l1', 'solver': 'saga'}
0.9002521286056621 {'C': 0.001, 'penalty': 'l2', 'solver': 'saga'}


{'C': 0.01, 'penalty': 'l1', 'solver': 'saga'}

In [0]:
cv_result = grid_search_clf.cv_results_
for mean_score, params in zip(cv_result['mean_test_score'], cv_result['params']):
  print(mean_score, params)
grid_search_clf.best_params_

0.9002521286056621 {'C': 10, 'penalty': 'l1', 'solver': 'saga'}
0.9002521286056621 {'C': 10, 'penalty': 'l2', 'solver': 'saga'}
0.9002521286056621 {'C': 1, 'penalty': 'l1', 'solver': 'saga'}
0.9002521286056621 {'C': 1, 'penalty': 'l2', 'solver': 'saga'}
0.9001470563977063 {'C': 0.1, 'penalty': 'l1', 'solver': 'saga'}
0.9002521286056621 {'C': 0.1, 'penalty': 'l2', 'solver': 'saga'}
0.901636178422903 {'C': 0.01, 'penalty': 'l1', 'solver': 'saga'}
0.9002521286056621 {'C': 0.01, 'penalty': 'l2', 'solver': 'saga'}
0.8949072862116091 {'C': 0.001, 'penalty': 'l1', 'solver': 'saga'}
0.9002521286056621 {'C': 0.001, 'penalty': 'l2', 'solver': 'saga'}


{'C': 0.01, 'penalty': 'l1', 'solver': 'saga'}

In [0]:
y_predict = grid_search_clf.predict(X_test)
cm = confusion_matrix(y_test, y_predict)
print(classification_report(y_test, y_predict, 
                            target_names=['0','1','2','3','4','5','6','7','8','9']))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96      1001
           1       0.93      0.97      0.95      1127
           2       0.90      0.88      0.89       991
           3       0.88      0.87      0.88      1032
           4       0.90      0.91      0.90       980
           5       0.88      0.81      0.84       863
           6       0.94      0.95      0.95      1014
           7       0.94      0.92      0.93      1070
           8       0.86      0.83      0.84       944
           9       0.86      0.90      0.88       978

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000



preprocessing comparision 

In [0]:
X_train, X_test, y_train, y_test = split_set()
print(X_train.shape)


lg_model  = LogisticRegression(C=0.1, penalty='l1', random_state = 77)
lg_model.fit(X_train,y_train)

y_predict = lg_model.predict(X_test)
cm = confusion_matrix(y_test, y_predict)
print(classification_report(y_test, y_predict, 
                            target_names=['0','1','2','3','4','5','6','7','8','9']))

(8000, 784)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


              precision    recall  f1-score   support

           0       0.94      0.96      0.95       196
           1       0.95      0.99      0.97       227
           2       0.86      0.86      0.86       206
           3       0.84      0.87      0.85       202
           4       0.89      0.91      0.90       196
           5       0.83      0.77      0.80       178
           6       0.91      0.91      0.91       192
           7       0.90      0.88      0.89       206
           8       0.84      0.83      0.84       195
           9       0.84      0.82      0.83       202

    accuracy                           0.88      2000
   macro avg       0.88      0.88      0.88      2000
weighted avg       0.88      0.88      0.88      2000



In [0]:
#X_train, X_test, y_train, y_test = split_set()
#print(X_train.shape)

X_train_reshape = X_train_reshape.reshape(10000,196)
lg_model  = LogisticRegression(C=0.1, penalty='l1', random_state = 77)
lg_model.fit(X_train_reshape,y_train)

y_predict = lg_model.predict(X_train_reshape)
cm = confusion_matrix(y_train, y_predict)
print(classification_report(y_test, y_predict, 
                            target_names=['0','1','2','3','4','5','6','7','8','9']))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       980
           1       0.11      1.00      0.20      1135
           2       0.00      0.00      0.00      1032
           3       0.00      0.00      0.00      1010
           4       0.00      0.00      0.00       982
           5       0.00      0.00      0.00       892
           6       0.00      0.00      0.00       958
           7       0.00      0.00      0.00      1028
           8       0.00      0.00      0.00       974
           9       0.00      0.00      0.00      1009

    accuracy                           0.11     10000
   macro avg       0.01      0.10      0.02     10000
weighted avg       0.01      0.11      0.02     10000



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
X_train_norm_re = X_train_norm_re.reshape(8000,784)
X_test_norm_re = X_test_norm_re.reshape(2000,784)

lg_model  = LogisticRegression(C=0.1, penalty='l1', random_state = 77)
lg_model.fit(X_train_norm_re,y_train)

y_predict = lg_model.predict(X_test_norm_re)
cm = confusion_matrix(y_test, y_predict)
print(classification_report(y_test, y_predict, 
                            target_names=['0','1','2','3','4','5','6','7','8','9']))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


              precision    recall  f1-score   support

           0       0.95      0.97      0.96       196
           1       0.90      0.98      0.94       227
           2       0.91      0.83      0.87       206
           3       0.91      0.88      0.89       202
           4       0.83      0.89      0.86       196
           5       0.91      0.80      0.85       178
           6       0.93      0.94      0.94       192
           7       0.91      0.91      0.91       206
           8       0.84      0.86      0.85       195
           9       0.83      0.84      0.84       202

    accuracy                           0.89      2000
   macro avg       0.89      0.89      0.89      2000
weighted avg       0.89      0.89      0.89      2000



In [0]:
#X_train_b_or_w
#X_test_b_or_w

lg_model  = LogisticRegression(C=0.001, 0.01, 0.1, 1, 10 penalty='l2', random_state = 42)
lg_model.fit(X_train_b_or_w,y_train)

y_predict = lg_model.predict(X_test_b_or_w)
cm = confusion_matrix(y_test, y_predict)
print(classification_report(y_test, y_predict, 
                            target_names=['0','1','2','3','4','5','6','7','8','9']))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


              precision    recall  f1-score   support

           0       0.95      0.97      0.96       196
           1       0.91      0.98      0.94       227
           2       0.92      0.84      0.88       206
           3       0.88      0.84      0.86       202
           4       0.85      0.89      0.87       196
           5       0.90      0.80      0.85       178
           6       0.92      0.94      0.93       192
           7       0.91      0.88      0.90       206
           8       0.79      0.87      0.83       195
           9       0.84      0.84      0.84       202

    accuracy                           0.89      2000
   macro avg       0.89      0.89      0.89      2000
weighted avg       0.89      0.89      0.89      2000



# **Show Score Matrix**

In [0]:
# NAMES
# grid_search_clf
# clf
# clf.s

y_predict = #MODEL NAME#.predict(X_test)
#NAME# = confusion_matrix(y_test, y_predict)
print(classification_report(y_test, y_predict, 
                            target_names = [str(range(0, 10))]))

# cv_result = #MODEL NAME#.cv_results_

# cvres = new_lg_model.best_params_  # 
# for mean_score, params in zip(cv_result['mean_test_score'], cv_result['params']):
#   print(np.sqrt(mean_score), params)    # '0.818125 {'C': 0.1, 'penalty': 'l1'}'

# **Show Score | Params**

In [0]:
#NAME#.best_estimator_

cv_result = #NAME#.cv_results_
# cv_result = #NAME#.best_params_
for mean_score, params in zip(cv_result['mean_test_score'], cv_result['params']):
  print(mean_score, params)